In [2]:
import pandas as pd
from pyhive import presto
from tqdm import tqdm
import re
import time
import os
import boto3
import os
import pandas as pd
import datetime
import time
import gzip
import s3fs
import sys
import datetime
from io import BytesIO, TextIOWrapper
from s3fs import S3FileSystem
from boto3 import client
from pyhive import presto
from pyhive import hive
import math
from datetime import datetime
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import *
import sklearn.svm as svm
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv1D, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from haversine import haversine

In [6]:
agg1 = pd.DataFrame()
agg2 = pd.DataFrame()
month = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for m in tqdm(month):
    with open(f"ord_dat_20{m}.pkl","rb") as fr:
        rider = pickle.load(fr)
    tmp1 = rider[['ord_no', 'cth_wk_code', 'ymd']].drop_duplicates().groupby(['cth_wk_code', 'ymd']).count().reset_index()
    tmp2 = rider[['ord_no', 'cth_wk_code', 'ymd', 'srv_amt', 'ord_amt', 'dvry_amt']].drop_duplicates().groupby(['cth_wk_code', 'ymd'])[['srv_amt', 'ord_amt', 'dvry_amt']].agg('sum', 'mean').reset_index()
    agg1 = pd.concat([agg1, tmp1], axis = 0)
    agg2 = pd.concat([agg2, tmp2], axis = 0)

100%|██████████| 12/12 [14:48<00:00, 74.07s/it] 


In [7]:
finl_total = pd.merge(agg1, agg2, how = 'inner', on = ['cth_wk_code', 'ymd'])

In [8]:
with open("summary2_rider.pkl","wb") as fw:
    pickle.dump(finl_total, fw)

In [10]:
with open(f"summary2_rider.pkl","rb") as fr:
        sum1 = pickle.load(fr)

In [9]:
with open(f"ord_dat_total_agg.pkl","rb") as fr:
        sum2 = pickle.load(fr)

In [11]:
sum1.dtypes

cth_wk_code    object
ymd             int64
ord_no          int64
srv_amt         int64
ord_amt         int64
dvry_amt        int64
dtype: object

In [12]:
sum2.dtypes

wk_code                         object
ymd                              int64
cth_date        min     datetime64[ns]
finish_date     max     datetime64[ns]
total_cth_time         timedelta64[ns]
dtype: object

In [13]:
sum1 = sum1.rename(columns = {'cth_wk_code':'wk_code'})

In [ ]:
finl_sum1 = pd.merge(sum1, sum2, )

In [3]:
with open(f"ord_dat_total_agg.pkl","rb") as fr:
        ord_21 = pickle.load(fr)

In [4]:
with open(f"ord_dat_total_agg20.pkl","rb") as fr:
        ord_20 = pickle.load(fr)

In [100]:
with open(f"wk_profit_total_2021.pkl","rb") as fr:
        pf_21 = pickle.load(fr)

In [101]:
with open(f"wk_profit_total_2020.pkl","rb") as fr:
        pf_20 = pickle.load(fr)

In [7]:
ord_21_new = pd.DataFrame(ord_21.values, columns = ['wk_code', 'ymd', 'cth_date', 'finish_date', 'total_cth_time'])
ord_20_new = pd.DataFrame(ord_20.values, columns = ['wk_code', 'ymd', 'cth_date', 'finish_date', 'total_cth_time'])

In [8]:
pf_21 = pf_21.reset_index(drop = True)
pf_20 = pf_20.reset_index(drop = True)

In [9]:
ord_21_new['ymd'] = ord_21_new.ymd.apply(lambda x : str(x))
ord_20_new['ymd'] = ord_20_new.ymd.apply(lambda x : str(x))

In [10]:
pf_21 = pf_21.rename(columns = {'cth_wk_code' : 'wk_code'})
pf_20 = pf_20.rename(columns = {'cth_wk_code' : 'wk_code'})

In [11]:
ord_21_new = pd.merge(ord_21_new, pf_21, how = 'left', on = ['wk_code', 'ymd'])
ord_20_new = pd.merge(ord_20_new, pf_20, how = 'left', on = ['wk_code', 'ymd'])

In [12]:
ord_21_new = ord_21_new.loc[pd.notnull(ord_21_new.cth_br_code), :]
ord_20_new = ord_20_new.loc[pd.notnull(ord_20_new.cth_br_code), :]

In [13]:
ord_21_new['total_cth_hour'] = round(ord_21_new['total_cth_time'].dt.seconds/3600, 2)

In [14]:
ord_20_new['total_cth_hour'] = round(ord_20_new['total_cth_time'].dt.seconds/3600, 2)

In [15]:
ord_21_new['ym'] = ord_21_new.ymd.apply(lambda x: x[0:6])
ord_20_new['ym'] = ord_20_new.ymd.apply(lambda x: x[0:6])

In [16]:
agg_ym_21 = ord_21_new.groupby(['wk_code', 'ym'])[['ymd']].count().reset_index()
agg_ym_21 = agg_ym_21.rename(columns = {'ymd':'day_cnt'})

In [17]:
agg_ym_20 = ord_20_new.groupby(['wk_code', 'ym'])[['ymd']].count().reset_index()
agg_ym_20 = agg_ym_20.rename(columns = {'ymd':'day_cnt'})

In [18]:
dt21_dat = ord_21_new[['ym', 'ymd']].drop_duplicates().groupby(['ym']).count().reset_index()
dt20_dat = ord_20_new[['ym', 'ymd']].drop_duplicates().groupby(['ym']).count().reset_index()

In [19]:
agg_ym_21 = pd.merge(agg_ym_21, dt21_dat, how = 'left', on = ['ym'])
agg_ym_20 = pd.merge(agg_ym_20, dt20_dat, how = 'left', on = ['ym'])

In [20]:
agg_ym_21['cth_per'] = round(agg_ym_21.day_cnt/agg_ym_21.ymd, 2)

In [21]:
agg_ym_20['cth_per'] = round(agg_ym_20.day_cnt/agg_ym_20.ymd, 2)

In [22]:
agg_ym_21['week_num'] = agg_ym_21.ymd.apply(lambda x : int(x/7))
agg_ym_20['week_num'] = agg_ym_20.ymd.apply(lambda x : int(x/7))

In [23]:
agg_ym_21['cth_day_per_week'] = agg_ym_21.day_cnt/agg_ym_21.week_num
agg_ym_20['cth_day_per_week'] = agg_ym_20.day_cnt/agg_ym_20.week_num

In [24]:
agg_ym_21 = agg_ym_21.rename(columns = {'ymd' : 'month_day'})
agg_ym_20 = agg_ym_20.rename(columns = {'ymd' : 'month_day'})

In [25]:
ord_21_new = pd.merge(ord_21_new, agg_ym_21[['wk_code', 'ym', 'cth_day_per_week']], how = 'left', on = ['wk_code', 'ym'])
ord_20_new = pd.merge(ord_20_new, agg_ym_20[['wk_code', 'ym', 'cth_day_per_week']], how = 'left', on = ['wk_code', 'ym'])

In [26]:
ord_21_new = pd.merge(ord_21_new, agg_ym_21[['wk_code', 'ym', 'day_cnt', 'month_day', 'cth_per']], how = 'left', on = ['wk_code', 'ym'])
ord_20_new = pd.merge(ord_20_new, agg_ym_20[['wk_code', 'ym', 'day_cnt', 'month_day', 'cth_per']], how = 'left', on = ['wk_code', 'ym'])

In [27]:
agg_ym_21 = pd.merge(agg_ym_21, ord_21_new.groupby(['wk_code', 'ym'])[['total_cth_hour']].sum().reset_index(), how = 'left', on = ['wk_code', 'ym'])

In [28]:
agg_ym_20 = pd.merge(agg_ym_20, ord_20_new.groupby(['wk_code', 'ym'])[['total_cth_hour']].sum().reset_index(), how = 'left', on = ['wk_code', 'ym'])

In [29]:
ord_21_new['start_time'] = ord_21_new.cth_date.dt.hour

In [30]:
ord_20_new['start_time'] = ord_20_new.cth_date.dt.hour

In [31]:
agg_ym_21['avg_hour_per_week'] = agg_ym_21.total_cth_hour/agg_ym_21.week_num
agg_ym_20['avg_hour_per_week'] = agg_ym_20.total_cth_hour/agg_ym_20.week_num

In [32]:
ord_21_new = pd.merge(ord_21_new, agg_ym_21[['wk_code', 'ym', 'avg_hour_per_week']], how = 'left', on = ['wk_code', 'ym'])
ord_20_new = pd.merge(ord_20_new, agg_ym_20[['wk_code', 'ym', 'avg_hour_per_week']], how = 'left', on = ['wk_code', 'ym'])

In [73]:
with open("yr21_data.pkl","wb") as fw:
    pickle.dump(ord_21_new, fw)


with open("yr20_data.pkl","wb") as fw:
    pickle.dump(ord_20_new, fw)

In [47]:
ord_21_new.loc[(ord_21_new.ymd == '20210101')&(ord_21_new.week_day == 5), :]

,wk_code,ymd,cth_date,finish_date,total_cth_time,cth_br_code,ord_cnt,cth_profit,wk_fee,wk_income,...,ord_kind,total_cth_hour,ym,cth_day_per_week,day_cnt,month_day,cth_per,start_time,avg_hour_per_week,week_day
133755,WA1809,20210101,2021-01-02 00:01:27,2021-01-02 01:00:23,0 days 00:58:56,B1720,1.0,3000.0,-300.0,2700.0,...,정상,0.98,202101,5.5,22,31,0.71,0,34.5425,5
156904,WA7710,20210101,2021-01-02 00:14:01,2021-01-02 06:52:48,0 days 06:38:47,B3481,1.0,3500.0,-400.0,3100.0,...,정상,6.65,202101,7.5,30,31,0.97,0,116.0850,5
196659,WB6965,20210101,2021-01-02 00:04:07,2021-01-02 00:31:08,0 days 00:27:01,B3254,2.0,7600.0,-600.0,7000.0,...,정상,0.45,202101,1.0,4,31,0.13,0,2.2850,5


In [55]:
def extr_weekday(col):
    datetime_str = col[0:4] + '-' + col[4:6] + '-' + col[6:] + ' 00:00:00'
    format = '%Y-%m-%d %H:%M:%S'
    tmp_date = datetime.datetime.strptime(datetime_str,format)
    return tmp_date.weekday()

In [56]:
ord_21_new['week_day'] = ord_21_new.apply(lambda x: extr_weekday(x['ymd']), axis = 1)
ord_20_new['week_day'] = ord_20_new.apply(lambda x: extr_weekday(x['ymd']), axis = 1)

In [58]:
ord_21_new.[['ym', 'ymd', 'week_day']].drop_duplicates()
ord_20_new.[['ym', 'ymd', 'week_day']].drop_duplicates()

SyntaxError: invalid syntax (1360466962.py, line 1)

In [62]:
week_day21 = ord_21_new.loc[(ord_21_new.week_day ==5)|(ord_21_new.week_day==6)][['ym', 'ymd', 'week_day']].drop_duplicates().groupby(['ym'])[['ymd']].count().reset_index()
week_day20 = ord_20_new.loc[(ord_20_new.week_day ==5)|(ord_20_new.week_day==6)][['ym', 'ymd', 'week_day']].drop_duplicates().groupby(['ym'])[['ymd']].count().reset_index()

In [64]:
week_wk_cnt21 = ord_21_new.loc[(ord_21_new.week_day ==5)|(ord_21_new.week_day==6)][['ym', 'wk_code', 'ymd']].groupby(['ym', 'wk_code']).count().reset_index()
week_wk_cnt20 = ord_20_new.loc[(ord_20_new.week_day ==5)|(ord_20_new.week_day==6)][['ym', 'wk_code', 'ymd']].groupby(['ym', 'wk_code']).count().reset_index()

In [66]:
week_wk_cnt21 = week_wk_cnt21.rename(columns = {'ymd':'week_cth_cnt'})

In [67]:
week_wk_cnt20 = week_wk_cnt20.rename(columns = {'ymd':'week_cth_cnt'})

In [68]:
week_wk_cnt21

,ym,wk_code,week_cth_cnt
0,202101,W00442,10
1,202101,W00695,10
2,202101,W01219,6
3,202101,W02719,7
4,202101,W02891,5
...,...,...,...
272581,202112,WL2617,1
272582,202112,WL2619,1
272583,202112,WL2621,1
272584,202112,WL2622,1


In [70]:
ord_21_new = pd.merge(ord_21_new, week_wk_cnt21, how = 'left', on = ['ym', 'wk_code'])
ord_20_new = pd.merge(ord_20_new, week_wk_cnt20, how = 'left', on = ['ym', 'wk_code'])

In [72]:
ord_21_new['weekend_per'] = round(ord_21_new.week_cth_cnt / ord_21_new.ord_cnt, 2)
ord_20_new['weekend_per'] = round(ord_20_new.week_cth_cnt / ord_20_new.ord_cnt, 2)

In [ ]:
with open("yr21_data.pkl","wb") as fw:
    pickle.dump(ord_21_new, fw)


with open("yr20_data.pkl","wb") as fw:
    pickle.dump(ord_20_new, fw)

In [3]:
with open("yr21_data.pkl","rb") as fr:
        ord_21_new = pickle.load(fr)

with open("yr20_data.pkl","rb") as fr:
        ord_20_new = pickle.load(fr)

In [4]:
ord_21_new

,wk_code,ymd,cth_date,finish_date,total_cth_time,cth_br_code,ord_cnt,cth_profit,wk_fee,wk_income,...,ym,cth_day_per_week,day_cnt,month_day,cth_per,start_time,avg_hour_per_week,week_day,week_cth_cnt,weekend_per
0,W00442,20210101,2021-01-01 11:55:46,2021-01-01 19:00:14,0 days 07:04:28,B0038,30.0,128601.0,-8700.0,119901.0,...,202101,6.50,26,31,0.84,11,65.4675,4,10.0,0.33
1,W00442,20210102,2021-01-02 11:42:22,2021-01-02 21:10:50,0 days 09:28:28,B0038,37.0,149818.0,-10500.0,139318.0,...,202101,6.50,26,31,0.84,11,65.4675,5,10.0,0.27
2,W00442,20210103,2021-01-03 11:48:24,2021-01-03 22:29:53,0 days 10:41:29,B0038,43.0,164907.0,-11700.0,153207.0,...,202101,6.50,26,31,0.84,11,65.4675,6,10.0,0.23
3,W00442,20210104,2021-01-04 12:08:28,2021-01-04 22:06:23,0 days 09:57:55,B0038,22.0,89030.0,-6300.0,82730.0,...,202101,6.50,26,31,0.84,12,65.4675,0,10.0,0.45
4,W00442,20210106,2021-01-06 11:38:13,2021-01-06 23:00:14,0 days 11:22:01,B0038,43.0,147251.0,-11100.0,136151.0,...,202101,6.50,26,31,0.84,11,65.4675,2,10.0,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5151162,WL3797,20211231,2021-12-31 18:10:38,2021-12-31 23:25:24,0 days 05:14:46,B2917,23.0,108700.0,-12629.0,96071.0,...,202112,0.25,1,31,0.03,18,1.3125,4,NaN,NaN
5151163,WL3809,20211231,2021-12-31 20:43:06,2021-12-31 22:41:57,0 days 01:58:51,B1396,10.0,31700.0,-3040.0,28660.0,...,202112,0.25,1,31,0.03,20,0.4950,4,NaN,NaN
5151164,WL3811,20211231,2021-12-31 20:13:58,2022-01-01 00:31:31,0 days 04:17:33,B2303,24.0,99647.0,-8800.0,90847.0,...,202112,0.25,1,31,0.03,20,1.0725,4,NaN,NaN
5151165,WL3818,20211231,2021-12-31 22:02:06,2021-12-31 23:41:06,0 days 01:39:00,B5137,9.0,40992.0,-2700.0,38292.0,...,202112,0.25,1,31,0.03,22,0.4125,4,NaN,NaN


In [87]:
ord_21_new.columns

Index(['wk_code', 'ymd', 'cth_date', 'finish_date', 'total_cth_time',
       'cth_br_code', 'ord_cnt', 'cth_profit', 'wk_fee', 'wk_income',
       'wk_income_tax', 'ord_kind', 'total_cth_hour', 'ym',
       'cth_day_per_week_x', 'day_cnt_x', 'month_day', 'cth_per_x',
       'start_time', 'avg_hour_per_week', 'week_day', 'week_cth_cnt',
       'weekend_per', 'avg_start_time', 'avg_ord_cnt', 'day_cnt_y',
       'cth_day_per_week_y', 'cth_per_y'],
      dtype='object')

In [4]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['start_time']].mean().reset_index().rename(columns = {'start_time' : 'avg_start_time'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['start_time']].mean().reset_index().rename(columns = {'start_time' : 'avg_start_time'}), how = 'left', on = ['wk_code'])

In [5]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['ord_cnt']].mean().reset_index().rename(columns = {'ord_cnt' : 'avg_ord_cnt'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['ord_cnt']].mean().reset_index().rename(columns = {'ord_cnt' : 'avg_ord_cnt'}), how = 'left', on = ['wk_code'])

In [6]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['day_cnt']].mean().reset_index().rename(columns = {'day_cnt' : 'avg_day_cnt'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['day_cnt']].mean().reset_index().rename(columns = {'day_cnt' : 'avg_day_cnt'}), how = 'left', on = ['wk_code'])

In [7]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['cth_day_per_week']].mean().reset_index().rename(columns = {'cth_day_per_week' : 'avg_cth_day_per_week'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['cth_day_per_week']].mean().reset_index().rename(columns = {'cth_day_per_week' : 'avg_cth_day_per_week'}), how = 'left', on = ['wk_code'])

In [8]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['cth_per']].mean().reset_index().rename(columns = {'cth_per' : 'avg_cth_per'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['cth_per']].mean().reset_index().rename(columns = {'cth_per' : 'avg_cth_per'}), how = 'left', on = ['wk_code'])

In [9]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['avg_hour_per_week']].mean().reset_index().rename(columns = {'avg_hour_per_week' : 'avg_avg_hour_per_week'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['avg_hour_per_week']].mean().reset_index().rename(columns = {'avg_hour_per_week' : 'avg_avg_hour_per_week'}), how = 'left', on = ['wk_code'])

In [10]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['total_cth_hour']].mean().reset_index().rename(columns = {'total_cth_hour' : 'avg_total_cth_hour'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['total_cth_hour']].mean().reset_index().rename(columns = {'total_cth_hour' : 'avg_total_cth_hour'}), how = 'left', on = ['wk_code'])

In [29]:
ord_21_new = pd.merge(ord_21_new, ord_21_new.groupby(['wk_code'])[['weekend_per']].mean().reset_index().rename(columns = {'weekend_per' : 'avg_weekend_per'}), how = 'left', on = ['wk_code'])
ord_20_new = pd.merge(ord_20_new, ord_20_new.groupby(['wk_code'])[['weekend_per']].mean().reset_index().rename(columns = {'weekend_per' : 'avg_weekend_per'}), how = 'left', on = ['wk_code'])

In [13]:
tmp21 = ord_21_new[['wk_code', 'avg_start_time', 'avg_ord_cnt', 'avg_day_cnt', 'avg_cth_day_per_week', 'avg_cth_per', 'avg_avg_hour_per_week', 'avg_total_cth_hour']].drop_duplicates().reset_index(drop = True)
tmp20 = ord_20_new[['wk_code', 'avg_start_time', 'avg_ord_cnt', 'avg_day_cnt', 'avg_cth_day_per_week', 'avg_cth_per', 'avg_avg_hour_per_week', 'avg_total_cth_hour']].drop_duplicates().reset_index(drop = True)

In [14]:
tmp21.describe()

,avg_start_time,avg_ord_cnt,avg_day_cnt,avg_cth_day_per_week,avg_cth_per,avg_avg_hour_per_week,avg_total_cth_hour
count,74248.000000,74248.000000,74248.000000,74248.000000,74248.000000,74248.000000,74248.000000
mean,13.653958,24.646398,14.843226,3.710806,0.487161,29.598043,6.939809
std,2.898635,14.535857,8.715034,2.178759,0.286444,23.544139,3.099556
min,0.000000,1.000000,1.000000,0.250000,0.030000,0.000000,0.000000
25%,11.250000,13.259259,6.600000,1.650000,0.216364,8.131484,4.590700
50%,13.142857,23.333333,16.044740,4.011185,0.527813,25.403811,7.163542
75%,15.888889,34.142857,22.615385,5.653846,0.742659,47.982093,9.388947
max,23.000000,128.026667,31.277333,7.819333,1.025973,162.288470,22.746731


In [28]:
ord_21_new.columns

Index(['wk_code', 'ymd', 'cth_date', 'finish_date', 'total_cth_time',
       'cth_br_code', 'ord_cnt', 'cth_profit', 'wk_fee', 'wk_income',
       'wk_income_tax', 'ord_kind', 'total_cth_hour', 'ym', 'cth_day_per_week',
       'day_cnt', 'month_day', 'cth_per', 'start_time', 'avg_hour_per_week',
       'week_day', 'week_cth_cnt', 'weekend_per', 'avg_start_time',
       'avg_ord_cnt', 'avg_day_cnt', 'avg_cth_day_per_week', 'avg_cth_per',
       'avg_avg_hour_per_week', 'avg_total_cth_hour'],
      dtype='object')

In [39]:
part1 = list(ord_21_new.loc[(ord_21_new.avg_start_time >= 19)&(ord_21_new.avg_total_cth_hour <= 4), :].wk_code.unique())
part2 = list(ord_21_new.loc[(ord_21_new.avg_avg_hour_per_week <= 9), :].wk_code.unique())
part3 = list(ord_21_new.loc[(ord_21_new.avg_cth_per <= 0.22), :].wk_code.unique())
part4 = list(ord_21_new.loc[(ord_21_new.avg_cth_day_per_week < 3), :].wk_code.unique())
part5 = list(ord_21_new.loc[(ord_21_new.avg_weekend_per > 0.9), :].wk_code.unique())

In [40]:
blk_ls = []
blk_ls.extend(part1)
blk_ls.extend(part2)
blk_ls.extend(part3)
blk_ls.extend(part4)
blk_ls.extend(part5)
new_blk = list(set(blk_ls))

In [87]:
part10 = list(ord_20_new.loc[(ord_20_new.avg_start_time >= 19)&(ord_20_new.avg_total_cth_hour <= 4), :].wk_code.unique())
part20 = list(ord_20_new.loc[(ord_20_new.avg_avg_hour_per_week <= 9), :].wk_code.unique())
part30 = list(ord_20_new.loc[(ord_20_new.avg_cth_per <= 0.22), :].wk_code.unique())
part40 = list(ord_20_new.loc[(ord_20_new.avg_cth_day_per_week < 3), :].wk_code.unique())
part50 = list(ord_20_new.loc[(ord_20_new.avg_weekend_per > 0.9), :].wk_code.unique())

In [88]:
blk_ls0 = []
blk_ls0.extend(part10)
blk_ls0.extend(part20)
blk_ls0.extend(part30)
blk_ls0.extend(part40)
blk_ls0.extend(part50)
new_blk0 = list(set(blk_ls0))

In [89]:
len(new_blk0)

24263

In [41]:
len(new_blk)

33169

In [99]:
len(ord_20_new.wk_code.unique())

54428

In [43]:
filt1 = ord_21_new.loc[~ord_21_new.wk_code.isin(new_blk), :]

In [77]:
wk_ls = list(filt1.wk_code.unique())

In [98]:
len(filt2.wk_code.unique())

30165

In [44]:
with open("new_wk_ls_dat.pkl","wb") as fw:
    pickle.dump(filt1, fw)

In [45]:
filt1.columns

Index(['wk_code', 'ymd', 'cth_date', 'finish_date', 'total_cth_time',
       'cth_br_code', 'ord_cnt', 'cth_profit', 'wk_fee', 'wk_income',
       'wk_income_tax', 'ord_kind', 'total_cth_hour', 'ym', 'cth_day_per_week',
       'day_cnt', 'month_day', 'cth_per', 'start_time', 'avg_hour_per_week',
       'week_day', 'week_cth_cnt', 'weekend_per', 'avg_start_time',
       'avg_ord_cnt', 'avg_day_cnt', 'avg_cth_day_per_week', 'avg_cth_per',
       'avg_avg_hour_per_week', 'avg_total_cth_hour', 'avg_weekend_per'],
      dtype='object')

In [78]:
filt2 = ord_20_new.loc[ord_20_new.wk_code.isin(wk_ls), :]

In [90]:
filt2 = ord_20_new.loc[~ord_20_new.wk_code.isin(new_blk0), :]

In [56]:
inform1 = filt1.groupby(['ym', 'wk_code'])[['cth_profit', 'wk_fee', 'wk_income', 'wk_income_tax']].sum().reset_index().groupby(['ym'])[['cth_profit', 'wk_income']].mean().reset_index()
inform2 = filt1[['ym', 'wk_code', 'day_cnt', 'month_day', 'cth_per']].drop_duplicates().groupby(['ym']).mean().reset_index()
inform3 = filt1[['ymd', 'ym', 'wk_code', 'ord_cnt']].drop_duplicates().groupby(['ym'])[['ord_cnt']].mean().reset_index()
inform4 = filt1[['ymd', 'ym', 'wk_code', 'total_cth_hour']].drop_duplicates().groupby(['ym'])[['total_cth_hour']].mean().reset_index()

In [91]:
inform12 = filt2.groupby(['ym', 'wk_code'])[['cth_profit', 'wk_fee', 'wk_income', 'wk_income_tax']].sum().reset_index().groupby(['ym'])[['cth_profit', 'wk_income']].mean().reset_index()
inform22 = filt2[['ym', 'wk_code', 'day_cnt', 'month_day', 'cth_per']].drop_duplicates().groupby(['ym']).mean().reset_index()
inform32 = filt2[['ymd', 'ym', 'wk_code', 'ord_cnt']].drop_duplicates().groupby(['ym'])[['ord_cnt']].mean().reset_index()
inform44 = filt2[['ymd', 'ym', 'wk_code', 'total_cth_hour']].drop_duplicates().groupby(['ym'])[['total_cth_hour']].mean().reset_index()

In [57]:
finl_inform21 = pd.merge(inform1, inform2, how = 'inner', on=['ym'])
finl_inform21 = pd.merge(finl_inform21, inform3, how = 'inner', on=['ym'])
finl_inform21 = pd.merge(finl_inform21, inform4, how = 'inner', on=['ym'])

In [92]:
finl_inform20 = pd.merge(inform12, inform22, how = 'inner', on=['ym'])
finl_inform20 = pd.merge(finl_inform20, inform32, how = 'inner', on=['ym'])
finl_inform20 = pd.merge(finl_inform20, inform44, how = 'inner', on=['ym'])

In [63]:
finl_inform21_new = round(finl_inform21, 2)

In [93]:
finl_inform20_new = round(finl_inform20, 2)

In [94]:
real_finl_dat = pd.concat([finl_inform20_new, finl_inform21_new], axis = 0)

In [95]:
real_finl_dat_new = pd.DataFrame(real_finl_dat.values, columns = ['년월', '수행수익', '라이더수익', '월별평균수행일수', '월별전체일수', '월별평균수행비율', '일평균수행건수', '일평균수행시간'])

In [97]:
real_finl_dat_new.to_excel('2020_2021_평균라이더수익데이터_동일조건제외.xlsx')

In [96]:
real_finl_dat_new

,년월,수행수익,라이더수익,월별평균수행일수,월별전체일수,월별평균수행비율,일평균수행건수,일평균수행시간
0,202001,2372239.99,2217854.71,20.56,31.0,0.66,32.89,8.71
1,202002,2354926.16,2200497.29,19.27,29.0,0.66,35.1,8.81
2,202003,2347170.46,2190267.8,20.51,31.0,0.66,33.01,8.66
3,202004,2235901.58,2087381.58,20.3,30.0,0.68,31.68,8.62
4,202005,2304718.75,2149712.02,20.39,31.0,0.66,32.35,8.75
5,202006,2241671.2,2090529.02,19.86,30.0,0.66,32.26,8.73
6,202007,2328227.2,2170475.38,20.41,31.0,0.66,32.35,8.75
7,202008,2677645.19,2496567.14,19.84,31.0,0.64,38.09,9.07
8,202009,2273237.04,2113643.51,19.42,30.0,0.65,32.52,8.67
9,202010,2127769.35,1975319.49,19.44,31.0,0.63,30.37,8.48


In [107]:
pf_20['ym'] = pf_20.ymd.apply(lambda x : x[0:6])
pf_21['ym'] = pf_21.ymd.apply(lambda x : x[0:6])

In [111]:
pf_20 = pf_20.reset_index(drop = True)
pf_21 = pf_21.reset_index(drop = True)

In [112]:
pf_20 = pf_20.loc[pf_20.ym != 202101, :]

In [113]:
pf_21 = pf_21.loc[pf_21.ym != 202201, :]

In [119]:
yr20_profit = pf_20.groupby(['ym', 'cth_wk_code'])[['wk_income']].sum().reset_index().groupby(['ym'])[['wk_income']].mean().reset_index()

In [123]:
yr20_profit['wk_income'] = yr20_profit['wk_income'].astype('int')

In [121]:
yr21_profit = pf_21.groupby(['ym', 'cth_wk_code'])[['wk_income']].sum().reset_index().groupby(['ym'])[['wk_income']].mean().reset_index()

In [124]:
yr21_profit['wk_income'] = yr21_profit['wk_income'].astype('int')

In [126]:
yr20_profit.to_excel('라이더수익_20년도.xlsx')
yr21_profit.to_excel('라이더수익_21년도.xlsx')